# Create bernCrop Dataset



In [ ]:
# load required modules
import cv2
import eodal
import os
import numpy as np
import h5py
import pandas as pd
import utm
import torch
from matplotlib.lines import Line2D
import seaborn as sns
import xml.etree.ElementTree as ET

from pathlib import Path
from eodal.core.sensors import Sentinel2
import geopandas as gpd
from shapely.geometry import Polygon
from eodal.config import get_settings

# make plots larger by default
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 15]

print('eodal version: {}'.format(eodal.__version__))

# we need to tell EOdal that we work using a local data source
settings = get_settings()
settings.USE_STAC = False


output_shapefile_path_BERN = f"../raw_data/LANDKULT/data/BERN_big_bbox.shp"
output_shapefile_path_tiles =f"../raw_data/LANDKULT/data/tiles/"
output_shapefile_path_tile_T32TLT = f"../raw_data/LANDKULT/data/tiles/T32TLT.shp"
output_shapefile_path_tile_T32TLS = f"../raw_data/LANDKULT/data/tiles/T32TLS.shp"
output_shapefile_path_tile_T32TMT = f"../raw_data/LANDKULT/data/tiles/T32TMT.shp"
output_shapefile_path_tile_T32TMS = f"../raw_data/LANDKULT/data/tiles/T32TMS.shp"
shapefile_path_landkult = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL.shp'
output_shapefile_path_landkult = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL_bern_bbox.shp'
output_shapefile_path_landkult_short = '../raw_data/LANDKULT/data/LANDKULT_NUTZFL_short_bern_bbox.shp'
output_shapefile_path_landkult_short_eodal = 'D:/Temp/AgroLuege/raw_data/LANDKULT/data/LANDKULT_NUTZFL_short_bern_bbox.shp'

In [ ]:
def convert_coordinates_to_utm(coordinates):
    utm_coordinates = []
    for lat, lon in coordinates:
        utm_coordinate = utm.from_latlon(lat, lon)
        utm_coordinates.append((utm_coordinate[0], utm_coordinate[1], utm_coordinate[2], utm_coordinate[3]))
    return utm_coordinates

def get_adjacent_points(utm_coordinates):
    adjacent_coordinates_beneath = []
    for easting, northing, zone_number, zone_letter in utm_coordinates:
        # Point beneath
        right_point = utm.to_latlon(easting - 100000, northing, zone_number, zone_letter)
        adjacent_coordinates_beneath.append(right_point)
    return adjacent_coordinates_beneath

def get_bern_bbox():
    # Define the coordinates
    x1, y1, x2, y2 = 361630.,5140066., 416830.,  5238466.

    # Create a GeoDataFrame with a single Point geometry
    geometry = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
    gdf_bern = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:32632")
    # Save the GeoDataFrame to a shapefile
    gdf_bern.to_file(output_shapefile_path_BERN)
    return gdf_bern


def get_tile_folder_path(data_dir):
    paths = []
    def get_subdirectories(path, depth=0, max_depth=0):
        if depth > max_depth:
            return

        subdirectories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        for subdir in subdirectories:
            subdir_path = os.path.join(path, subdir)
            if subdir_path.endswith('.SAFE'):
                paths.append(subdir_path)
            get_subdirectories(subdir_path, depth + 1, max_depth)
    get_subdirectories(data_dir)
    # sort the paths by timestamp
    paths.sort()
    return paths


def make_polygon_within(polygon1, polygon2):
    return  polygon1.intersection(polygon2)


def read_tile_data_from_safe(tile_paths, coords, tile, band_selection = ['B02', 'B03', 'B04', 'B08'], polygon_size = 240*240):
    # TODO: all file paths please
    tile_shape_path = output_shapefile_path_tiles+f'{tile}.shp'
    tile_gdf = gpd.read_file(tile_shape_path)
    for path_SAFE in tile_paths[0:2]:
        vector_feature_data = []
        for i,coordinates in coords.iterrows():
            x1, y1, x2, y2 = coordinates.iloc[:4].tolist()
            polygon = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
            polygon_gdf = gpd.GeoDataFrame(geometry=[polygon],crs='EPSG:32632')

            polygon_clipped_coords = make_polygon_within(tile_gdf.geometry[0],polygon_gdf.geometry[0])
            polygon_clipped_coords_df = gpd.GeoDataFrame(geometry=[polygon_clipped_coords],crs='EPSG:32632')
            intersection_area = polygon_clipped_coords_df.geometry.iloc[0].area

            if intersection_area == polygon_size:
                # print(f'started bounding_box: {coordinates} with index:{i} at timestamp {path_SAFE}')
                # print("Intersection Area:", polygon_clipped_coords_df.geometry.iloc[0].area)

                try:
                    # read data from .SAFE dataset for the selected AOI and spectral bands
                    handler = Sentinel2.from_safe(
                        in_dir=Path(path_SAFE),
                        vector_features=polygon_clipped_coords_df,
                        band_selection=band_selection,
                        apply_scaling=False
                    )
                except ValueError as ex:
                    print(f'error:\n{ex} but continue anyway\n Path:{path_SAFE}')
                    #save wrong polygons
                    polygon_clipped_coords_df.to_csv(f'./errors/error_polygon_{i}_{tile}.csv')
                    continue
                # ignore the value if its blackfilled
                if handler.is_blackfilled == True:
                    print(f"Skip is blackfilled: {path_SAFE}")
                    polygon_clipped_coords_df

                    continue
                # first resample the spectral bands using bicubic interpolation
                handler.resample(
                    target_resolution=10,
                    interpolation_method=cv2.INTER_NEAREST_EXACT,
                    inplace=True
                )

                # create a numpy array and remove last band
                #TODO: check if this is okay, we select 24x24 but the outcome is 26x26
                # timestamp_tile_data = [handler.to_xarray().to_numpy()[0:4,0:24,0:24]]
                single_feature_data = np.array(handler.to_xarray().to_numpy()[0:4])
                vector_feature_data.append(single_feature_data)
                # save tile data
            else:
                # print("Intersection Area:", polygon_clipped_coords_df.geometry.iloc[0].area)
                # print('edge case intersection not fully, is therefore ignored for this tile.')
                continue
            # save after each timestamp
        
        if len(vector_feature_data) > 0:
            vector_feature_data = np.array(vector_feature_data)
            print(vector_feature_data.shape)
            save_tile_data(vector_feature_data, tile)
            save_ground_truth_data(i,tile)

def save_tile_data(tile_array, tile,dataset_data_name="data",add_axis=True):
    file_name_tile = f'../raw_data/BernCrop/tiles/{tile}.hdf5'
    if add_axis:
        tile_array=tile_array[np.newaxis,:]
    data_shape = tile_array.shape
    print(data_shape)


    with h5py.File(file_name_tile, 'a') as hf:
        # Check if the dataset already exists
        if dataset_data_name in hf:
            dataset = hf[dataset_data_name]
        else:
            dtype = "float32"  # Use the appropriate data type for your data
            dataset = hf.create_dataset(dataset_data_name, shape=(0,) + data_shape[1:], dtype=dtype, maxshape=(None,) + data_shape[1:])
            
        current_size = dataset.shape[0]
        new_size = current_size + tile_array.shape[0]
        # Resize the dataset to accommodate the new batch
        dataset.resize(new_size, axis=0)
        # Append the new batch to the dataset
        dataset[current_size:new_size, :] = tile_array

def save_label_data(index_data,tile):
    # Specify the file path
    file_path = r'..\raw_data\BernCrop\tensor_label.pt'
    # Load the tensor from the file
    all_labels = torch.load(file_path)
    data_label = all_labels[index_data]

    save_tile_data(data_label,tile,'gt')

def save_field_label_data(index_data,tile):
    # Specify the file path
    file_path = r'..\raw_data\BernCrop\tensor_field.pt'

    # Load the tensor from the file
    all_labels = torch.load(file_path)
    data_label = all_labels[index_data]

    save_tile_data(data_label,tile,'gt_instance')

def save_ground_truth_data(index_data,tile):

    save_label_data(index_data,tile)
    save_field_label_data(index_data,tile)
    

def read_tile_data(tile,dataset_data_name="data"):
    filename_tile = f'../raw_data/BernCrop/tiles/{tile}.hdf5'
    # Open the HDF5 file in read mode
    with h5py.File(filename_tile, "r") as file:
        # Check if the "data" dataset exists in the file
        if dataset_data_name in file:
            # Access the dataset and read its contents into a NumPy array
            dataset = file[dataset_data_name][:]
        else:
            print(f"Dataset {dataset_data_name} not found in the HDF5 file.")
    return dataset

def get_coords(tile=None,is_short=False):
    if tile == None:
        coords = pd.read_csv(r'..\raw_data\BernCrop\bboxes_sentinel_24x24.csv',index_col=None)
        coords= coords.iloc[:, 1:5]
    else:
        if is_short:
            coords = pd.read_csv(fr'..\raw_data\BernCrop\bboxes_sentinel_24x24_{tile}_short.csv',index_col=None)
        else:
            coords = pd.read_csv(fr'..\raw_data\BernCrop\bboxes_sentinel_24x24_{tile}.csv',index_col=None)
        coords= coords.iloc[:, 1:5]
    return coords


def stack_sample_tiles(tile_array,minimal_t=10,):
    if minimal_t != None:
        tile_array = np.transpose(tile_array, (0, 1, 3, 4,2))
        sampled_indices = np.random.choice(tile_array.shape[1], size=minimal_t, replace=False)
        tile_array = tile_array[:,sampled_indices]
        print(tile_array.shape)
    # tile_array_return = np.expand_dims(tile_array, axis=0)
    print(tile_array.shape)
    return tile_array



In [ ]:
coordinates_T32TLT = [(6.3279, 46.8356), (7.8166, 46.8356), (7.8166, 47.8474), (6.3279, 47.8474)]
polygon_T32TLT = Polygon(coordinates_T32TLT)
gdf_T32TLT = gpd.GeoDataFrame(geometry=[polygon_T32TLT],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TLT.to_file(output_shapefile_path_tile_T32TLT)

utm_coordinates_T32TLS = convert_coordinates_to_utm(coordinates_T32TLT)
coordinates_T32TLS = get_adjacent_points(utm_coordinates_T32TLS)
polygon_T32TLS = Polygon(coordinates_T32TLS)
gdf_T32TLS = gpd.GeoDataFrame(geometry=[polygon_T32TLS],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TLS.to_file(output_shapefile_path_tile_T32TLS)

coordinates_T32TMT = [(7.6629, 46.8582), (9.1305, 46.8582), (9.1305, 47.8536), (7.6629, 47.8536)]
polygon_T32TMT = Polygon(coordinates_T32TMT)
gdf_T32TMT = gpd.GeoDataFrame(geometry=[polygon_T32TMT],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TMT.to_file(output_shapefile_path_tile_T32TMT)

utm_coordinates_T32TMS= convert_coordinates_to_utm(coordinates_T32TMT)
coordinates_T32TMS = get_adjacent_points(utm_coordinates_T32TMS)
polygon_T32TMS = Polygon(coordinates_T32TMS)
gdf_T32TMS = gpd.GeoDataFrame(geometry=[polygon_T32TMS],crs="EPSG:4326").to_crs(crs="EPSG:32632")
gdf_T32TMS.to_file(output_shapefile_path_tile_T32TMS)

In [ ]:
# tiles = ['T32TLT','T32TLS','T32TMT','T32TMS']
# for tile in tiles:
#     tile_shape_path = output_shapefile_path_tiles+f'{tile}.shp'
#     tile_gdf = gpd.read_file(tile_shape_path,crs='EPSG:32632')

#     coords = pd.read_csv(r'..\raw_data\BernCrop\bboxes_sentinel_24x24.csv',index_col=[0])
#     coords.sort_index(ascending=True)
#     geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, y1,x2,y2 in zip(coords['x1'], coords['y1'],coords['x2'], coords['y2'])]
#     gdf = gpd.GeoDataFrame(coords, geometry=geometry)
#     completely_inside_polygons = gdf[gdf.geometry.within(tile_gdf.geometry.iloc[0])]
#     completely_inside_polygons = completely_inside_polygons[completely_inside_polygons.geometry.within(get_bern_bbox().geometry.iloc[0])]
#     completely_inside_polygons.to_csv(fr'../raw_data/BernCrop/bboxes_sentinel_24x24_{tile}.csv')


In [ ]:
# for tile in tiles:
#     tile_paths = get_tile_folder_path(Path('E:/S2_Data_CH22/' + str(tile)))
#     for i in tile_paths:
#         # Path to your XML file
#         xml_file_path = i+"\INSPIRE.xml"
#         print(xml_file_path)

#         # Parse the XML file
#         tree = ET.parse(xml_file_path)
#         root = tree.getroot()

#         # Define the namespace dictionary
#         namespace = {
#             "gmd": "http://www.isotc211.org/2005/gmd",
#             "gco": "http://www.isotc211.org/2005/gco"
#         }

#         # Define the XPath expression with namespaces
#         xpath_expression = "./gmd:identificationInfo/gmd:MD_DataIdentification/gmd:abstract/gco:CharacterString"

#         # Find the element using find() with the namespace
#         abstract_element = root.find(xpath_expression, namespaces=namespace)

#         # Extract the text content
#         abstract_text = abstract_element.text if abstract_element is not None else None

#         abstract_text= abstract_text.replace(' ',',')
#         coordinates_list_str = abstract_text.split(',')
#         coordinates_list_str = [coord for coord in coordinates_list_str if coord]
#         coordinates_list = [(float(coordinates_list_str[i]), float(coordinates_list_str[i + 1])) for i in range(0, len(coordinates_list_str), 2)]
#         coordinates_list
#         polygon = Polygon(coordinates_list)

#         gdf = gpd.GeoDataFrame(geometry=[polygon])
#         gdf.plot()
#         plt.show()
#         plt.close()

In [ ]:
# coords= get_coords()
# # Create a GeoPandas GeoDataFrame
# geometry = [Polygon([(row['x1'], row['y1']), (row['x2'], row['y1']),
#                 (row['x2'], row['y2']), (row['x1'], row['y2'])])
#         for _, row in coords.iterrows()]

# gdf_polygons = gpd.GeoDataFrame(geometry=geometry)

# # Create a figure and axis
# fig, ax = plt.subplots(figsize=(24, 15))

# # Plot GeoDataFrames
# gdf_T32TMT.plot(ax=ax, color='#3d5a80', edgecolor='black', alpha=0.5)
# gdf_T32TMS.plot(ax=ax, color='#98c1d9', edgecolor='black', alpha=0.5)
# gdf_T32TLS.plot(ax=ax, color='#e0fbfc', edgecolor='black', alpha=0.5)
# gdf_T32TLT.plot(ax=ax, color='#293241', edgecolor='black', alpha=0.5)
# get_bern_bbox().plot(ax=ax, color='#ee6c4d', edgecolor='black', alpha=0.5)
# gdf_polygons.plot(ax=ax, color='black', edgecolor='black', alpha=0.7)

# ax.set_title('UTM-Tiles and Bern Bounding-Box')

# # Create a custom legend using Line2D for GeoPandas plots
# legend_elements = [
#     Line2D([0], [0], color='#3d5a80', label='T32TMT'),
#     Line2D([0], [0], color='#98c1d9', label='T32TMS'),
#     Line2D([0], [0], color='#e0fbfc', label='T32TLS'),
#     Line2D([0], [0], color='#293241', label='T32TLT'),
#     Line2D([0], [0], color='#ee6c4d', label='Bern B-Box'),
#     Line2D([0], [0], color='black',label='240m x 240m Polygons')
# ]

# # Add the legend outside the plot
# ax.legend(handles=legend_elements, loc='upper right')

# # Show the plot
# plt.show()


In [ ]:
# read tile data and save to dhf5 files
tiles = ['T32TLS']#,'T32TLT','T32TMS','T32TMT']
# tiles = ['T32TLT','T32TLS','T32TMT','T32TMS']


In [ ]:
# TODO: 
# E:\S2_Data_CH22\T32TLS\S2B_MSIL2A_20221225T103349_N0509_R108_T32TLS_20221225T114808.SAFE\INSPIRE.xml -> gut 
# E:\S2_Data_CH22\T32TLT\S2A_MSIL2A_20220812T103031_N0400_R108_T32TLT_20220812T182800.SAFE\INSPIRE.xml -> gut
# E:\S2_Data_CH22\T32TMS\S2A_MSIL2A_20221127T102351_N0400_R065_T32TMS_20221127T140108.SAFE\INSPIRE.xml -> gut
# E:\S2_Data_CH22\T32TMT\S2A_MSIL2A_20220603T102611_N0400_R108_T32TMT_20220603T170511.SAFE\INSPIRE.xml -> gut

In [ ]:
for tile in tiles:
# for tile in tiles:
    print(f"Start {tile}")
    coords = get_coords(tile,True)
    data_dir = Path('E:/S2_Data_CH22/' + tile)
    tile_paths = get_tile_folder_path(data_dir)
    read_tile_data_from_safe(tile_paths,coords,tile)
    print(f"End {tile}")

In [ ]:
read_tile_data('T32TLS','data').shape

In [ ]:
# read_tile_data('T32TMT','data').shape

In [ ]:
for i,v in enumerate(tiles):
        tile = read_tile_data(tiles[i])
        tile_labels = read_tile_data(tiles[i],'gt')
        tile_field_labels = read_tile_data(tiles[i],'gt_instance')

        print(tile_labels.shape)
        #TODO: change minimal t
        save_tile_data(stack_sample_tiles(tile,12),'BernCrop',add_axis=False)
        save_tile_data(stack_sample_tiles(tile_labels,None),'BernCrop','gt',add_axis=False)
        save_tile_data(stack_sample_tiles(tile_field_labels,None),'BernCrop','gt_instance',add_axis=False)

In [ ]:
read_tile_data('BernCrop','data').shape

In [ ]:
read_tile_data('BernCrop','gt').shape

In [ ]:
read_tile_data('BernCrop','gt_instance').shape